In [ ]:
import pandas as pd
import unicodedata
import re

def normalize_name(name):
    if not isinstance(name, str) or name == 'N/A':
        return None
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    name = name.lower().strip()
    name = re.sub(r'\s+', ' ', name)
    return name

df['normalized_player'] = df['player'].apply(normalize_name)
df_final['normalized_player'] = df_final['player_name'].apply(normalize_name)

df_final_subset = df_final[['normalized_player', 'price', 'skill/pot']]

df_merge = df.merge(df_final_subset, 
                    on='normalized_player', 
                    how='inner')

df_merge['minutes'] = df_merge['minutes'].str.replace(',', '').astype(float)
df_merge = df_merge[df_merge['minutes'] > 900]

df_merge = df_merge.drop(columns=['normalized_player'])
cols = [col for col in df_merge.columns if col not in ['price', 'skill/pot']] + ['price', 'skill/pot']
df_merge = df_merge[cols]

df_merge.to_csv('df_merge.csv', index=False)

In [ ]:
import pandas as pd
import unicodedata
import re

def normalize_name(name):
    if not isinstance(name, str) or name == 'N/A':
        return None
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    name = name.lower().strip()
    name = re.sub(r'\s+', ' ', name)
    return name

df = pd.read_csv('results.csv', header=[0,1,2])
df_final = pd.read_csv('football_transfers_players.csv')

df.columns = ['_'.join([str(c) for c in col if c]).strip('_') for col in df.columns]
player_col = [col for col in df.columns if col.endswith('_Player')][0]
minutes_col = [col for col in df.columns if col.endswith('_minutes')][0]

df['normalized_player'] = df[player_col].apply(normalize_name)
df_final['normalized_player'] = df_final['player_name'].apply(normalize_name)

df_final_subset = df_final[['normalized_player', 'price', 'skill/pot']]

df_merge = df.merge(df_final_subset, on='normalized_player', how='inner')

df_merge[minutes_col] = df_merge[minutes_col].str.replace(',', '').astype(float)
df_merge = df_merge[df_merge[minutes_col] > 900]

df_merge = df_merge.drop(columns=['normalized_player'])
cols = [col for col in df_merge.columns if col not in ['price', 'skill/pot']] + ['price', 'skill/pot']
df_merge = df_merge[cols]

df_merge.to_csv('merged-players.csv', index=False)